In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict

In [2]:
load_dotenv(override=True)

True

In [3]:
INSTRUCTIONS = (
    "You are a research assistant. Given a search term, you search the web for that term and "
    "produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 "
    "words. Capture the main points. Write succintly, no need to have complete sentences or good "
    "grammar. This will be consumed by someone synthesizing a report, so its vital you capture the "
    "essence and ignore any fluff. Do not include any additional commentary other than the summary "
    "itself."
)

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool()],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [4]:
async def main():
    message = "Latest AI Agent frameworks in 2025"

    with trace("Search"):
        result = await Runner.run(search_agent, message)
        print(result.final_output)

In [5]:
await main()

In 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent systems. LangChain facilitates the creation of applications powered by large language models (LLMs), integrating with multiple LLMs and supporting semantic search and API interactions. LangGraph builds upon LangChain, focusing on multi-agent systems with coordination tools and visual graph-based workflows. CrewAI emphasizes teamwork through a role-based architecture, enabling specialized agents to collaborate effectively. Microsoft's Semantic Kernel integrates traditional development tools with AI capabilities, supporting multiple languages and complex workflows. AutoGen, also from Microsoft, automates the generation of code and models for complex workflows, offering a modular architecture and human-in-the-loop capabilities. OpenAI Gym provides diverse environments for training reinforcement learning agents, while Rasa offers an open-source framework for developing customized

In [5]:
HOW_MANY_SEARCHES = 5

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."


class WebSearchItem(BaseModel):
    reason: str
    "Your reasoning for why this search is important to the query."

    query: str
    "The search term to use for the web search."


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """A list of web searches to perform to best answer the query."""


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [8]:
async def main():
    message = "Latest AI Agent frameworks in 2025"

    with trace("Search"):
        result = await Runner.run(planner_agent, message)
        print(result.final_output)

In [21]:
await main()

searches=[WebSearchItem(reason='To find the newest frameworks related to AI agents released in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To identify popular AI agent frameworks and their features as of 2025.', query='top AI agent frameworks 2025'), WebSearchItem(reason='To gather information about advancements in AI agent technologies in 2025.', query='AI agent technology advancements 2025'), WebSearchItem(reason='To explore research papers or articles discussing AI agent frameworks developed in 2025.', query='research papers AI agent frameworks 2025'), WebSearchItem(reason='To find comparisons of different AI agent frameworks introduced in 2025.', query='comparison of AI agent frameworks 2025'), WebSearchItem(reason='To look for reviews or case studies on the application of AI agent frameworks in 2025.', query='AI agent framework case studies 2025'), WebSearchItem(reason='To find conferences or events in 2025 that focused on AI agent frameworks.', query='

In [9]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com")
    to_email = To("ed.donner@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content)
    mail_json = mail.get()
    response = sg.client.mail.send.post(request_body=mail_json)
    return {"status": "success"}

In [10]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)



In [11]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str
    """A short 2-3 sentence summary of the findings."""

    markdown_report: str
    """The final report"""

    follow_up_questions: list[str]
    """Suggested topics to research further"""


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [12]:
class ResearchManager:

    async def run(self, query: str) -> None:
        trace_id = gen_trace_id()
        with trace("Research trace", trace_id=trace_id):
            print(f"View trace: https://platform.openai.com/traces/{trace_id}")
            print("Starting research...")

            search_plan = await self._plan_searches(query)
            search_results = await self._perform_searches(search_plan)
            report = await self._write_report(query, search_results)
            await self._send_email(report)
        

    async def _plan_searches(self, query: str) -> WebSearchPlan:
        print("Planning searches...")
        result = await Runner.run(
            planner_agent,
            f"Query: {query}",
        )
        print(f"Will perform {len(result.final_output.searches)} searches")
        return result.final_output_as(WebSearchPlan)

    async def _perform_searches(self, search_plan: WebSearchPlan) -> list[str]:
        print("Searching...")
        num_completed = 0
        tasks = [asyncio.create_task(self._search(item)) for item in search_plan.searches]
        results = []
        for task in asyncio.as_completed(tasks):
            result = await task
            if result is not None:
                results.append(result)
            num_completed += 1
            print(f"Searching... {num_completed}/{len(tasks)} completed")
        print("Finished searching")
        return results

    async def _search(self, item: WebSearchItem) -> str | None:
        input = f"Search term: {item.query}\nReason for searching: {item.reason}"
        try:
            result = await Runner.run(
                search_agent,
                input,
            )
            return str(result.final_output)
        except Exception:
            return None

    async def _write_report(self, query: str, search_results: list[str]) -> ReportData:
        print("Thinking about report...")
        input = f"Original query: {query}\nSummarized search results: {search_results}"
        result = await Runner.run(
            writer_agent,
            input,
        )

        print("Finished writing report")
        return result.final_output_as(ReportData)
    
    async def _send_email(self, report: ReportData) -> None:
        print("Writing email...")
        result = await Runner.run(
            email_agent,
            report.markdown_report,
        )
        print("Email sent")
        return report

In [ ]:
await ResearchManager().run("Latest AI Agent frameworks in 2025")

View trace: https://platform.openai.com/traces/trace_0cced55424f544ac9c159c71ee1655ac
Starting research...
Planning searches...
Will perform 10 searches
Searching...
Searching... 1/10 completed
Searching... 2/10 completed
Searching... 3/10 completed
Searching... 4/10 completed
Searching... 5/10 completed
Searching... 6/10 completed
Searching... 7/10 completed
Searching... 8/10 completed
Searching... 9/10 completed
Searching... 10/10 completed
Finished searching
Thinking about report...
Finished writing report
Writing email...
